In [1084]:
import pandas as pd
import numpy as np

import os
from pathlib import Path

import json


In [1233]:
#useful to look at the number of unqiue labels and find tickers with no matches
def unique_label_list(statement,m_dict):
    gaap_list = []
    for ticker in m_dict:
        if statement in m_dict[ticker] and m_dict[ticker][statement] is not None:
            current_statement = list(m_dict[ticker][statement].keys())
            gaap_list = np.concatenate((gaap_list,list(m_dict[ticker][statement].keys())))
            #print(f'{ticker}: {len(current_statement)}')
        else:
            print(ticker)

    gaap_list = np.unique(np.array(gaap_list).flatten())
    print(f"Number of unique labels {len(gaap_list)}")
    return gaap_list

def gaap_dict_with_tag_as_key(statement, m_dict):
    gaap_dict = {}
    for ticker in m_dict:

        if (statement in m_dict[ticker]) and (m_dict[ticker][statement] is not None):
            for tag in m_dict[ticker][statement].keys():
                if (tag in m_dict[ticker][statement]) and (m_dict[ticker][statement][tag] is not None):
                    if tag not in gaap_dict:
                        gaap_dict[tag] = dict()

                    for label in m_dict[ticker][statement][tag]:
                        if label not in gaap_dict[tag]:
                            gaap_dict[tag][label] = 1
                        else:
                            gaap_dict[tag][label] += 1


    return gaap_dict

def gaap_dict_with_label_as_key(gaap_dict):
    inverted_gaap_dict = {}
    for tag in gaap_dict:
        for label in gaap_dict[tag]:
            if gaap_dict[tag][label] > max(2,label_max):

                if label not in inverted_gaap_dict:

                    inverted_gaap_dict[label] = {}

                inverted_gaap_dict[label][tag] = gaap_dict[tag][label] 
    return inverted_gaap_dict

def formated_gaap_dict(inverted_gaap_dict):
    formated_inverted_gaap_dict = {}
    for label in inverted_gaap_dict:
        formated_inverted_gaap_dict[label] = np.array([])
        tag_list = []
        to_dict = {}
        for tag in inverted_gaap_dict[label]:
            matches = inverted_gaap_dict[label][tag]
            if matches not in to_dict:
                to_dict[matches] = np.array([])
            to_dict[matches] = np.append(to_dict[matches],tag)
            
        for matches in (sorted(to_dict.keys(),reverse=True)):
            formated_inverted_gaap_dict[label] = np.append(formated_inverted_gaap_dict[label],to_dict[matches])
            formated_inverted_gaap_dict[label] = list(formated_inverted_gaap_dict[label])

    return formated_inverted_gaap_dict


def create_formated_gaap_dict(data_path):
    
    mapping_dict = {}
    
    with open(f"{data_path}mappings/ticker_tag_label_mapping.json") as json_file:
        m_dict = json.load(json_file)
    
    print(f"Number of companies: {len(m_dict.keys())}")
    
    for statement in ['Income Statement','Cash Flow','Balance Sheet']:

        gaap_dict = gaap_dict_with_tag_as_key(statement, m_dict)
        inverted_gaap_dict = gaap_dict_with_label_as_key(gaap_dict)
        formated_inverted_gaap_dict = formated_gaap_dict(inverted_gaap_dict)
        
        
        with open(f"{data_path}mappings/placeholder_label_tag_mapping_{statement}.json","w") as json_file:
            json.dump(formated_inverted_gaap_dict,json_file)

        
        mapping_dict[statement] = formated_inverted_gaap_dict
        
        print_duplicates(mapping_dict[statement],statement)
    
    return mapping_dict
    
def print_duplicates(mapping_dict,statement):
    duplicate_dict = {}
    for label in mapping_dict:
        for tag in mapping_dict[label]:
            if tag not in duplicate_dict:
                duplicate_dict[tag] = []

            duplicate_dict[tag].append(label)
        duplicate_dict[tag] = list(np.unique(duplicate_dict[tag]))

    for tag in duplicate_dict:
        if len(duplicate_dict[tag]) >= 2:
            print(f"{statement} {tag}: {', '.join(duplicate_dict[tag])}")


In [1234]:
data_path = '../data/'
mapping_dict = create_formated_gaap_dict(data_path)

Number of companies: 149
Income Statement us-gaap_revenues: Gross Profit, Revenue
Income Statement us-gaap_interestexpense: Cost of Revenue, Interest Expense (Operating), Non-operating Interest Expenses
Income Statement us-gaap_incomelossfromcontinuingoperationsbeforeincometaxesextraordinaryitemsnoncontrollinginterest: EBT, Operating Income
Income Statement us-gaap_incomelossfromcontinuingoperationsbeforeincometaxesminorityinterestandincomelossfromequitymethodinvestments: EBT, Operating Income
Income Statement us-gaap_operatingincomeloss: EBIT, Operating Income
Income Statement us-gaap_netincomeloss: Income after Tax, Income from Continuous Operations, Consolidated Net Income/Loss, Net Income Common
Income Statement us-gaap_profitloss: Consolidated Net Income/Loss, Income after Tax, Income from Continuous Operations, Net Income Common
Income Statement us-gaap_incomelossfromcontinuingoperationsincludingportionattributabletononcontrollinginterest: Income after Tax, Income from Continuous

In [1369]:
with open(f"{data_path}mappings/canonical_label_tag_mapping.json") as json_file:
    m_dict = json.load(json_file)

In [1620]:
def add_standard_label(df_timeseries):
    df_timeseries_tmp = df_timeseries
    df_timeseries_tmp = df_timeseries_tmp.loc[:,[max(df_timeseries_tmp.columns)]].dropna() #only want to match against most recent month
    df_timeseries_tmp['standard_label'] = None

    for standard_label in m_dict['Income Statement']:
        for tag in m_dict['Income Statement'][standard_label]:
            if tag in df_timeseries_tmp.index.get_level_values(1).str.extract(r'([A-Za-z-_]+)___')[0].tolist():#checking for segment                  
                segment_slice = pd.Series(list(tag == df_timeseries_tmp.index.get_level_values(1).str.extract(r'([-A-Za-z_]+)___')[0]),index=df_timeseries_tmp.index)
                df_timeseries_tmp.loc[segment_slice,'standard_label'] = pd.Series(list(standard_label + ' ' + df_timeseries_tmp.loc[segment_slice,:].index.get_level_values(0).str.extract(r'(\([ A-Za-z\[\]]+\))')[0]),index=df_timeseries_tmp.loc[segment_slice,:].index)

            if tag in df_timeseries_tmp.index.get_level_values(1):
                label = df_timeseries_tmp[df_timeseries_tmp.index.get_level_values(1) == tag].index.get_level_values(0)[0]
                if label is not None:
                    df_timeseries_tmp.loc[(label,tag), 'standard_label'] = standard_label 
                    break #only want to add the first match
            
    return merge_new_label_and_correct_index(df_timeseries,df_timeseries_tmp['standard_label'])

def merge_new_label_and_correct_index(df_timeseries,s_standard_label):
    df_timeseries = df_timeseries.merge(s_standard_label,left_index=True,right_index=True,how="left")
    df_timeseries['filing_label'] = df_timeseries.index.get_level_values(0)
    df_timeseries['xbrl_tag'] = df_timeseries.index.get_level_values(1)
    df_timeseries = df_timeseries.set_index(['filing_label','xbrl_tag','standard_label'])
    return df_timeseries

In [1621]:
df_timeseries = pd.read_csv(f'{data_path}timeseries/AAPL/statement/CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS Unaudited.csv',index_col=[0,1])
df_timeseries = add_standard_label(df_timeseries)

In [1622]:
df_timeseries

,,,2020-03-28,2019-12-28,2019-09-28,2019-06-29,2019-03-30,2018-12-29,2018-09-29,2018-06-30,2018-03-31,2017-12-30,2017-09-30,2017-07-01,2017-04-01,2016-12-31,2016-09-24,2016-06-25,2016-03-26,2015-12-26,2015-09-26,2015-06-27,2015-03-28,2014-12-27,2014-09-27,2014-06-28,2014-03-29,2013-12-28,2013-09-28,2013-06-29,2013-03-30,2012-12-29,2012-09-29,2012-06-30,2012-03-31,2011-12-31,2011-09-24,2011-06-25,2011-03-26,2010-12-25,2010-09-25,2010-06-26,2010-03-27,2009-12-26,2009-09-26,2009-06-27,2009-03-28,2008-12-27,2008-09-27,2008-06-28,2007-09-29
filing_label,xbrl_tag,standard_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Revenue from Contract with Customer, Excluding Assessed Tax",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,Revenue,5.831300e+10,9.181900e+10,6.404000e+10,5.380900e+10,5.801500e+10,8.431000e+10,6.290000e+10,5.326500e+10,6.113700e+10,8.829300e+10,NaN,4.540800e+10,5.289600e+10,NaN,4.685200e+10,4.235800e+10,5.055700e+10,7.587200e+10,5.150100e+10,4.960500e+10,5.801000e+10,7.459900e+10,4.212300e+10,3.743200e+10,4.564600e+10,5.759400e+10,3.747200e+10,3.532300e+10,4.360300e+10,5.451200e+10,NaN,3.502300e+10,3.918600e+10,NaN,2.827000e+10,2.857100e+10,2.466700e+10,2.674100e+10,2.034300e+10,1.570000e+10,1.349900e+10,1.568300e+10,7.236000e+09,8.337000e+09,9.084000e+09,1.188000e+10,NaN,7.464000e+09,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Product [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,Revenue (Product [Member]),4.496500e+10,7.910400e+10,5.152900e+10,4.235400e+10,4.656500e+10,7.343500e+10,5.230100e+10,4.309500e+10,5.128700e+10,7.916400e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Service [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,Revenue (Service [Member]),1.334800e+10,1.271500e+10,1.251100e+10,1.145500e+10,1.145000e+10,1.087500e+10,1.059900e+10,1.017000e+10,9.850000e+09,9.129000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of sales,us-gaap_costofgoodsandservicessold,Cost of revenue,3.594300e+10,5.660200e+10,3.972700e+10,3.358200e+10,3.619400e+10,5.227900e+10,3.881600e+10,3.284400e+10,3.771500e+10,5.438100e+10,NaN,2.792000e+10,3.230500e+10,NaN,2.903900e+10,2.625200e+10,3.063600e+10,4.544900e+10,3.095300e+10,2.992400e+10,3.435400e+10,4.485800e+10,2.611400e+10,2.269700e+10,2.769900e+10,3.574800e+10,2.360100e+10,2.229900e+10,2.725400e+10,3.345200e+10,NaN,2.002900e+10,2.062200e+10,NaN,1.689000e+10,1.664900e+10,1.444900e+10,1.644300e+10,1.283100e+10,9.564000e+09,7.874000e+09,9.272000e+09,5.253000e+09,5.314000e+09,5.457000e+09,7.373000e+09,NaN,4.864000e+09,NaN
Cost of sales (Product [Member]),us-gaap_costofgoodsandservicessold___us-gaap_productmember,Cost of revenue (Product [Member]),3.132100e+10,5.207500e+10,3.523800e+10,2.947300e+10,3.204700e+10,4.823800e+10,3.469700e+10,2.895600e+10,3.393600e+10,5.057500e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of sales (Service [Member]),us-gaap_costofgoodsandservicessold___us-gaap_servicemember,Cost of revenue (Service [Member]),4.622000e+09,4.527000e+09,4.489000e+09,4.109000e+09,4.147000e+09,4.041000e+09,4.119000e+09,3.888000e+09,3.779000e+09,3.806000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit,us-gaap_grossprofit,Gross income,2.237000e+10,3.521700e+10,2.431300e+10,2.022700e+10,2.182100e+10,3.203100e+10,2.408400e+10,2.042100e+10,2.342200e+10,3.391200e+10,NaN,1.748800e+10,2.059100e+10,NaN,1.781300e+10,1.6106

In [1594]:
df_timeseries = pd.read_csv(f'{data_path}timeseries/AAPL/statement/CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS Unaudited.csv',index_col=[0,1])
df_timeseries['standard_label'] = None


tag = 'us-gaap_revenuefromcontractwithcustomerexcludingassessedtax'
standard_label = 'Revenue'

segment_slice = pd.Series(list(tag == df_timeseries.index.get_level_values(1).str.extract(r'([-A-Za-z_]+)___')[0]),index=df_timeseries.index)


In [1595]:
df_timeseries

,,2020-03-28,2019-12-28,2019-09-28,2019-06-29,2019-03-30,2018-12-29,2018-09-29,2018-06-30,2018-03-31,2017-12-30,2017-09-30,2017-07-01,2017-04-01,2016-12-31,2016-09-24,2016-06-25,2016-03-26,2015-12-26,2015-09-26,2015-06-27,2015-03-28,2014-12-27,2014-09-27,2014-06-28,2014-03-29,2013-12-28,2013-09-28,2013-06-29,2013-03-30,2012-12-29,2012-09-29,2012-06-30,2012-03-31,2011-12-31,2011-09-24,2011-06-25,2011-03-26,2010-12-25,2010-09-25,2010-06-26,2010-03-27,2009-12-26,2009-09-26,2009-06-27,2009-03-28,2008-12-27,2008-09-27,2008-06-28,2007-09-29,standard_label
label,xbrl_tag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Revenue from Contract with Customer, Excluding Assessed Tax",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,5.831300e+10,9.181900e+10,6.404000e+10,5.380900e+10,5.801500e+10,8.431000e+10,6.290000e+10,5.326500e+10,6.113700e+10,8.829300e+10,NaN,4.540800e+10,5.289600e+10,NaN,4.685200e+10,4.235800e+10,5.055700e+10,7.587200e+10,5.150100e+10,4.960500e+10,5.801000e+10,7.459900e+10,4.212300e+10,3.743200e+10,4.564600e+10,5.759400e+10,3.747200e+10,3.532300e+10,4.360300e+10,5.451200e+10,NaN,3.502300e+10,3.918600e+10,NaN,2.827000e+10,2.857100e+10,2.466700e+10,2.674100e+10,2.034300e+10,1.570000e+10,1.349900e+10,1.568300e+10,7.236000e+09,8.337000e+09,9.084000e+09,1.188000e+10,NaN,7.464000e+09,NaN,None
"Revenue from Contract with Customer, Excluding Assessed Tax (Product [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,4.496500e+10,7.910400e+10,5.152900e+10,4.235400e+10,4.656500e+10,7.343500e+10,5.230100e+10,4.309500e+10,5.128700e+10,7.916400e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Revenue (Product [Member])
"Revenue from Contract with Customer, Excluding Assessed Tax (Service [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,1.334800e+10,1.271500e+10,1.251100e+10,1.145500e+10,1.145000e+10,1.087500e+10,1.059900e+10,1.017000e+10,9.850000e+09,9.129000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Revenue (Service [Member])
Cost of sales,us-gaap_costofgoodsandservicessold,3.594300e+10,5.660200e+10,3.972700e+10,3.358200e+10,3.619400e+10,5.227900e+10,3.881600e+10,3.284400e+10,3.771500e+10,5.438100e+10,NaN,2.792000e+10,3.230500e+10,NaN,2.903900e+10,2.625200e+10,3.063600e+10,4.544900e+10,3.095300e+10,2.992400e+10,3.435400e+10,4.485800e+10,2.611400e+10,2.269700e+10,2.769900e+10,3.574800e+10,2.360100e+10,2.229900e+10,2.725400e+10,3.345200e+10,NaN,2.002900e+10,2.062200e+10,NaN,1.689000e+10,1.664900e+10,1.444900e+10,1.644300e+10,1.283100e+10,9.564000e+09,7.874000e+09,9.272000e+09,5.253000e+09,5.314000e+09,5.457000e+09,7.373000e+09,NaN,4.864000e+09,NaN,None
Cost of sales (Product [Member]),us-gaap_costofgoodsandservicessold___us-gaap_productmember,3.132100e+10,5.207500e+10,3.523800e+10,2.947300e+10,3.204700e+10,4.823800e+10,3.469700e+10,2.895600e+10,3.393600e+10,5.057500e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
Cost of sales (Service [Member]),us-gaap_costofgoodsandservicessold___us-gaap_servicemember,4.622000e+09,4.527000e+09,4.489000e+09,4.109000e+09,4.147000e+09,4.041000e+09,4.119000e+09,3.888000e+09,3.779000e+09,3.806000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
Gross Profit,us-gaap_grossprofit,2.237000e+10,3.521700e+10,2.431300e+10,2.022700e+10,2.182100e+10,3.203100e+10,2.408400e+10,2.042100e+10,2.342200e+10,3.391200e+10,NaN,1.748800e+10,2.059100e+10,NaN,1.781300e+10,1.610600e+10,1.992100e+10,3.042300e+10,2.054800e+10,1.968100e+10,2.365600e+10,2.974100e+10,1.600900e

MultiIndex([('Revenue from Contract with Customer, Excluding Assessed Tax (Product [Member])', ...),
            ('Revenue from Contract with Customer, Excluding Assessed Tax (Service [Member])', ...)],
           names=['label', 'xbrl_tag'])

In [1549]:
tag = 'us-gaap_revenuefromcontractwithcustomerexcludingassessedtax'


,0
0,Revenue (Product [Member])
1,Revenue (Service [Member])


In [1509]:
df_timeseries.index.get_level_values(1).str.extract(r'([-A-Za-z_]+)___')[0]df_timeseries.index.get_level_values(1).str.extract(r'([-A-Za-z_]+)___')[0]

0                                                   NaN
1     us-gaap_revenuefromcontractwithcustomerexcludi...
2     us-gaap_revenuefromcontractwithcustomerexcludi...
3                                                   NaN
4                    us-gaap_costofgoodsandservicessold
5                    us-gaap_costofgoodsandservicessold
6                                                   NaN
7                                                   NaN
8                                                   NaN
9                                                   NaN
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [1442]:
tag

'us-gaap_preferredstockdividendsincomestatementimpact'